In [1]:
#test emissions budgets following remap
import xarray as xr
import pandas as pd
import numpy as np

# Load the NetCDF file -- Fe emissions PI 
Rathod_OG = xr.open_dataset("BioFuel_1750_SRathod2017.nc")
Rathod_remapped = xr.open_dataset("BioFuel_1750_SRathod2017_0.9x1.25_remapcon.nc")

print("this worked")

this worked


In [2]:
print("OG", Rathod_OG) 

print("Remap", Rathod_remapped)

OG <xarray.Dataset> Size: 112kB
Dimensions:  (lon: 144, lat: 96)
Coordinates:
  * lon      (lon) float32 576B 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 384B -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
Data variables:
    SFEBF    (lat, lon) float32 55kB ...
    TFEBF    (lat, lon) float32 55kB ...
Attributes:
    Title:    Sol and Tot Fe emissions from BF in 1750 
Remap <xarray.Dataset> Size: 446kB
Dimensions:  (lon: 288, lat: 192)
Coordinates:
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
Data variables:
    SFEBF    (lat, lon) float32 221kB ...
    TFEBF    (lat, lon) float32 221kB ...
Attributes:
    CDI:          Climate Data Interface version 2.4.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Title:        Sol and Tot Fe emissions from BF in 1750 
    history:      Sun Apr 06 12:17:14 2025: cdo remapcon,grid_288x192.txt B

In [3]:
# Extract emissions values 
SFEBF = Rathod_remapped['SFEBF'] # soluble fraction Emission data
TFEBF = Rathod_remapped['TFEBF'] # insoluble fraction Emission data

Tot_FEBF = SFEBF + TFEBF 

print(Tot_FEBF)

<xarray.DataArray (lat: 192, lon: 288)> Size: 221kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0


In [4]:
# FIND AREA OF INDIVIDUAL GRID CELLS --------------------------------
lon = Rathod_remapped['lon'].values  # Longitude values 
lat = Rathod_remapped['lat'].values  # Latitude values

lon_rads = np.radians(lon)
lat_rads = np.radians(lat)
d_lat = np.abs(lat[1] - lat[0])  # Latitude grid spacing in degrees
d_lon = np.abs(lon[1] - lon[0])  # Longitude grid spacing in degrees
g_lat = np.radians(d_lat / 2)  # Latitude half-spacing in radians
g_lon = np.radians(d_lon / 2)  # Longitude half-spacing in radians

R = 6.3781E6 
cell_areas_staggered = []
for i in range(len(lat)):
    for j in range(len(lon)):
        lat_center = lat_rads[i]
        lon_center = lon_rads[j]
        lat_north = lat_center + g_lat
        lat_south = lat_center - g_lat
        lat_north = np.clip(lat_north, -np.pi / 2, np.pi / 2)
        lat_south = np.clip(lat_south, -np.pi / 2, np.pi / 2)
        area = R**2 * (np.sin(lat_north) - np.sin(lat_south)) * (2 * g_lon)
        cell_areas_staggered.append(area)

cell_areas_staggered = np.array(cell_areas_staggered).reshape(len(lat), len(lon))

# Verify to see if areas add to 5.1E14 in m^2
sum_sa_earth = cell_areas_staggered.sum()
print(f"surface area, {sum_sa_earth:3e}") 

all_FE_emiss = xr.Dataset({
    "Tot_FEBF": Tot_FEBF,
    "S_FEBF": SFEBF,
    })

# add cell area to original xarrays for easy calling 
all_FE_emiss['cell_area'] = xr.DataArray(
    cell_areas_staggered,
    dims=["lat", "lon"],  # Same dimensions as in the original dataset
    coords={"lat": all_FE_emiss['lat'], "lon": all_FE_emiss['lon']},  # Use original coordinates
    attrs={
        "units": "m^2",  # Specify units for the cell area
        "description": "Calculated grid cell area using staggered grid approach",
    },
)

surface area, 5.112020e+14


In [5]:
# Loop over all variables in the dataset
global_budgets = {}
individual_cell_emissions = {}

for var_name in all_FE_emiss.data_vars:
    if "FE" in var_name:  
        # Calculate individual emissions (for each cell) and convert from sec to annual and from kg to Tg
        individual_cell_emissions[var_name] = (all_FE_emiss[var_name] * all_FE_emiss['cell_area'] * 3600 * 24 * 365 *1E-9)
        
       # Calculate the total budget by summing individual emissions
        total_budget = float(individual_cell_emissions[Tot_FEBF].sum().values)  # Ensure it's a float
        
        # Store the budgets for the variable in a nested dictionary
        global_budgets[var_name] = {
            "Total_Budget": total_budget,
        }

budget_df = pd.DataFrame(global_budgets).T  # Transpose to get variables as rows and budgets as columns

# Reset index and give the proper column name for variables
budget_df.reset_index(inplace=True)
budget_df.rename(columns={'index': 'Variable'}, inplace=True)

# Ensure all columns have the correct numeric type
budget_df = budget_df.apply(pd.to_numeric, errors='ignore')  # Apply to convert all numeric

print("Fe emiss budget remapped", budget_df)

TypeError: unhashable type: 'DataArray'

In [6]:
# Loop over all variables in the dataset
global_budgets = {}
individual_cell_emissions = {}

# Compute emissions
individual_cell_emissions["Tot_FEBF"] = (
    all_FE_emiss["Tot_FEBF"] * all_FE_emiss["cell_area"] * 3600 * 24 * 365 * 1E-9
)

individual_cell_emissions["S_FEBF"] = (
    all_FE_emiss["S_FEBF"] * all_FE_emiss["cell_area"] * 3600 * 24 * 365 * 1E-9
)

# Calculate budgets
total_budget = float(individual_cell_emissions["Tot_FEBF"].sum().values)
sFe_budget = float(individual_cell_emissions["S_FEBF"].sum().values)

# Store
global_budgets = {
    "Tot_FEBF": {
        "Total_Budget": total_budget,
        "sFe_Budget": sFe_budget,
    }
}
budget_df = pd.DataFrame(global_budgets).T  # Transpose to get variables as rows and budgets as columns

# Reset index and give the proper column name for variables
budget_df.reset_index(inplace=True)
budget_df.rename(columns={'index': 'Variable'}, inplace=True)

# Ensure all columns have the correct numeric type
budget_df = budget_df.apply(pd.to_numeric, errors='ignore')  # Apply to convert all numeric

print("Fe emiss budget remapped", budget_df)

Fe emiss budget remapped    Variable  Total_Budget  sFe_Budget
0  Tot_FEBF       0.00079    0.000072


/glade/derecho/scratch/heplaas/tmp/ipykernel_8554/1334438251.py:32: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  budget_df = budget_df.apply(pd.to_numeric, errors='ignore')  # Apply to convert all numeric
